In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# GCP上的端到端机器学习：MLOps阶段6：使用Vertex AI Prediction开始使用TensorFlow Serving

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_tf_serving_tabular.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_tf_serving_tabular.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage6/get_started_with_tf_serving_tabular.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>        
</table>
<br/><br/><br/>

## 概览

本教程演示了如何使用`TensorFlow Serving`服务二进制，从`Vertex AI 端点`为一个非 TF 表格模型提供预测。

### 目标

在本教程中，您将学习如何在`Vertex AI Endpoint`资源上使用`Vertex AI Prediction`和`TensorFlow Serving` Serving二进制文件。

本教程使用以下谷歌云ML服务和资源：

- `Vertex AI Prediction`
- `Vertex AI Batch Prediction`
- `Vertex AI Models`
- `Vertex AI Endpoints`

执行的步骤包括：

- 下载一个预训练的TensorFlow表格模型。
- 将TensorFlow模型上传为`Vertex AI Model`资源。
- 创建一个`Endpoint`资源。
- 使用`TensorFlow Serving` Serving二进制文件将`Model`资源部署到`Endpoint`资源中。
- 向部署到`Endpoint`资源中的`Model`资源实例进行在线预测。
- 向`Model`资源实例进行批量预测。

### 模型

这个教程使用的模型是一个预训练的TensorFlow模型，该模型是在[波士顿房价数据集](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html)上训练的。这个教程使用的数据集版本已经集成到了TensorFlow中。训练好的模型预测房屋的中位价格，单位为千美元。

费用

本教程使用Google Cloud的计费组件：

- Vertex AI
- 云存储

了解[Vertex AI的定价](https://cloud.google.com/vertex-ai/pricing)和[云存储的定价](https://cloud.google.com/storage/pricing)，并使用[Pricing计算器](https://cloud.google.com/products/calculator/)根据您的预期使用情况生成费用估算。

## 安装

安装以下软件包以执行此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG -q
! pip3 install --upgrade google-cloud-pipeline-components $USER_FLAG -q
! pip3 install tensorflow-hub $USER_FLAG -q

重新启动内核

在安装了额外的包之后，您需要重新启动笔记本内核，以便它能够找到这些包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在开始之前

### GPU运行时

*如果有这个选项，请确保在GPU运行时中运行此笔记本。在Colab中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置您的谷歌云项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐号时，您将获得用于支付计算/存储成本的$300免费信用。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下API：Vertex AI APIs、Compute Engine APIs和Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您正在本地运行此笔记本，则需要安装[Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK为本笔记本中的所有命令使用正确的项目。

**注意**：Jupyter以`!`为前缀的行作为shell命令运行，并插补以`$`为前缀的Python变量。

#### 设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

### 区域

您也可以更改“REGION”变量，该变量在整个笔记本的其余部分中使用。 以下是Vertex AI支持的区域。 我们建议您选择最靠近您的区域。

- 美洲： `us-central1`
- 欧洲： `europe-west4`
- 亚太： `asia-east1`

您不能使用多区域存储桶进行Vertex AI培训。 并非所有区域都支持所有Vertex AI服务。

了解更多关于[Vertex AI区域](https：//cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

UUID

如果您正在进行实时教程会话，您可能正在使用共享测试账户或项目。为了避免用户之间在创建的资源上发生名称冲突，您为每个实例会话创建一个 UUID，并将其附加到您在本教程中创建的资源名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 验证您的Google Cloud账号

**如果您正在使用Vertex AI Workbench笔记本**，您的环境已经经过身份验证。请跳过这一步。

**如果您正在使用Colab**，运行下面的单元格，并按提示进行身份验证，通过oAuth验证您的账户。

**否则**，请按照以下步骤操作：

在Cloud控制台中，转到[创建服务账号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务账号**。

在**服务账号名称**字段中输入名称，然后点击**创建**。

在**授予此服务账号对项目的访问权限**部分，点击角色下拉列表。在过滤框中输入“Vertex”，然后选择**Vertex管理员**。在过滤框中输入“存储对象管理员”，然后选择**存储对象管理员**。

点击创建。包含您密钥的JSON文件将下载到您的本地环境。

在下面的单元格中，将您的服务账号密钥路径作为GOOGLE_APPLICATION_CREDENTIALS变量输入，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用什么笔记本环境，都需要执行以下步骤。**

当您为 Python 初始化 Vertex AI SDK 时，您需要指定一个云存储暂存桶。暂存桶是与您的数据集和模型资源相关的所有数据在各个会话之间保留的地方。

在下面设置您的云存储桶的名称。云存储桶的名称必须在所有 Google Cloud 项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + UUID
    BUCKET_URI = "gs://" + BUCKET_NAME

只有当您的存储桶尚不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查其内容验证对云存储桶的访问。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip

### 初始化用于 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化 Python 版的 Vertex AI SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

#### 设置硬件加速器

您可以为预测设置硬件加速器。

设置变量`DEPLOY_GPU/DEPLOY_NGPU`以使用支持GPU的容器映像，并指定分配给虚拟机（VM）实例的GPU数量。例如，要使用一个GPU容器映像，并为每个VM分配4个 Nvidia Telsa K80 GPU，您可以指定：

    （aip.gapic.AcceleratorType.NVIDIA_TESLA_K80, 4）

否则，指定`(None, None)`来使用一个在CPU上运行的容器映像。

了解有关[您所在地区的硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)的更多信息。

*注意*：在TF 2.3之前的GPU支持版本将无法在本教程中加载自定义模型。这是一个已知问题，并在TF 2.3中得到修复。这是由在serving函数中生成的静态图操作所导致的。如果您在自己的自定义模型上遇到此问题，请使用支持GPU的TF 2.3的容器映像。

In [ ]:
DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### 设置机器类型

接下来，设置用于预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置用于预测的VM的计算资源。
- `机器类型`
     - `n1-standard`：每个vCPU 3.75GB内存。
     - `n1-highmem`：每个vCPU 6.5GB内存。
     - `n1-highcpu`：每个vCPU 0.9GB内存。
 - `vCPUs`：\[2, 4, 8, 16, 32, 64, 96 \]中的数量。

*注意: 您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", DEPLOY_COMPUTE)

In [ ]:
! gcloud services enable artifactregistry.googleapis.com

## 创建一个私有的Docker存储库

您的第一步是在Google Artifact Registry中创建自己的Docker存储库。

1. 运行`gcloud artifacts repositories create`命令，在您的地区创建一个带有描述“docker存储库”的新Docker存储库。

2. 运行`gcloud artifacts repositories list`命令来验证您的存储库是否已创建。

In [ ]:
PRIVATE_REPO = "my-docker-repo"

! gcloud artifacts repositories create {PRIVATE_REPO} --repository-format=docker --location={REGION} --description="Docker repository"

! gcloud artifacts repositories list

### 配置私有仓库的身份验证

在推送或拉取容器映像之前，配置Docker使用`gcloud`命令行工具对您所在地区的`Artifact Registry`进行身份验证请求。

In [ ]:
! gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet

#### 用于提供服务的容器（Docker）映像

设置用于提供预测的 TensorFlow Serving Docker 容器映像。

1. 从 Docker Hub 拉取对应的 CPU 或 GPU Docker 映像以用于 TF Serving。
2. 创建一个标签用于在 Artifact Registry 中注册映像。
3. 将映像注册到 Artifact Registry。

了解更多关于[TensorFlow Serving](https://www.tensorflow.org/tfx/serving/docker)的信息。

In [ ]:
# Executes in Vertex AI Workbench
if DEPLOY_GPU:
    DEPLOY_IMAGE = (
        f"{REGION}-docker.pkg.dev/"
        + PROJECT_ID
        + f"/{PRIVATE_REPO}"
        + "/tf_serving:gpu"
    )
    TF_IMAGE = "tensorflow/serving:2.5.4-gpu"
else:
    DEPLOY_IMAGE = (
        f"{REGION}-docker.pkg.dev/"
        + PROJECT_ID
        + f"/{PRIVATE_REPO}"
        + "/tf_serving:cpu"
    )
    TF_IMAGE = "tensorflow/serving:2.5.4"

if not IS_COLAB:
    if DEPLOY_GPU:
        ! sudo docker pull tensorflow/serving:2.5.4-gpu
    else:
        ! sudo docker pull tensorflow/serving:2.5.4

    ! docker tag $TF_IMAGE $DEPLOY_IMAGE
    ! docker push $DEPLOY_IMAGE
else:
    # install docker daemon
    ! apt-get -qq install docker.io

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

在Colab中执行

In [ ]:
%%bash -s $IS_COLAB $DEPLOY_IMAGE $TF_IMAGE
if [ $1 == "False" ]; then
  exit 0
fi
set -x
dockerd -b none --iptables=0 -l warn &
for i in $(seq 5); do [ ! -S "/var/run/docker.sock" ] && sleep 2 || break; done
docker pull $3
docker tag tensorflow/serving $2
docker push $2
kill $(jobs -p)

## 获取预训练模型

为了演示目的，本教程使用了一个预训练的 TensorFlow 表格模型，然后将其上传到 `Vertex AI Model` 资源。一旦您拥有了 `Vertex AI Model` 资源，该模型就可以部署到 `Vertex AI Endpoint` 资源。

### 下载模型工件

接下来，将模型工件保存到一个 Cloud Storage 位置。

*注意:* 对于 TF Serving，MODEL_DIR 必须以一个数字结尾的子文件夹，例如，1。

In [ ]:
MODEL_ARTIFACTS = (
    "gs://cloud-samples-data/vertex-ai/model-deployment/models/boston/model"
)

MODEL_DIR = BUCKET_URI + "/model/1"

! gsutil cp -r {MODEL_ARTIFACTS} {MODEL_DIR}

上传模型以供服务

接下来，您将把您的 TensorFlow 模型上传到“Vertex AI 模型注册”服务，这将为您的自定义模型创建一个 Vertex AI “模型”资源。

将预训练的TensorFlow模型上传到“Vertex AI Model”资源中

最后，将TensorFlow模型的模型工件上传到“Vertex AI Model”资源中。由于您正在使用非Google预构建的Serving二进制文件 -- 即TensorFlow Serving，您需要指定以下额外的Serving配置设置：

- `serving_container_command`：要启动的Serving二进制文件（HTTP服务器）。
- `serving_container_args`：要传递给Serving二进制文件的参数。对于TensorFlow Serving，所需的参数包括：
    - `--model_name`：要分配给模型的可读名称。
    - `--model_base_name`：要在容器中存储模型工件的位置。Vertex服务将变量$(AIP_STORAGE_URI)设置为服务在容器中安装模型工件的位置。
    - `--rest_api_port`：要发送基于REST的预测请求的端口。可以是8080或8501（TensorFlow Serving的默认值）。
    - `--port`：要发送基于gRPC的预测请求的端口。对于TensorFlow Serving，应为8500。
- `serving_container_health_route`：定期向其发送ping请求以验证Serving二进制文件正在运行的服务的URL。对于TensorFlow Serving，这将是/v1/models/<model_name>。
- `serving_container_predict_route`：服务将路由基于REST的预测请求的URL。对于TF Serving，这将是/v1/models/[model_name]:predict。
- `serving_container_ports`：HTTP服务器监听请求的端口列表。

将模型上传到Vertex Model资源会返回一个长时间运行的操作，因为可能需要一些时间。

*注意：* 从模型路径中删除结尾的数字子文件夹（例如，/1）以便上传。Vertex服务将上传包含模型工件的子文件夹上方的父文件夹 -- 这正是TensorFlow Serving二进制文件所期望的。

*注意：* 当您将模型工件上传到“Vertex AI Model”资源时，您需要指定相应的部署容器镜像。

In [ ]:
MODEL_NAME = "boston_" + UUID

model = aip.Model.upload(
    display_name="boston_" + UUID,
    artifact_uri=MODEL_DIR[:-2],
    serving_container_image_uri=DEPLOY_IMAGE,
    serving_container_health_route="/v1/models/" + MODEL_NAME,
    serving_container_predict_route="/v1/models/" + MODEL_NAME + ":predict",
    serving_container_command=["/usr/bin/tensorflow_model_server"],
    serving_container_args=[
        "--model_name=" + MODEL_NAME,
        "--model_base_path=" + "$(AIP_STORAGE_URI)",
        "--rest_api_port=8080",
        "--port=8500",
        "--file_system_poll_wait_seconds=31540000",
    ],
    serving_container_ports=[8080],
)

print(model)

## 创建 `Endpoint` 资源

您可以使用 `Endpoint.create()` 方法来创建一个 `Endpoint` 资源。至少，您需要为端点指定显示名称。可选地，您可以指定项目和位置（地区）；否则，设置将继承您使用 `init()` 方法初始化 Vertex AI SDK 时设置的值。

在这个示例中，指定了以下参数：

- `display_name`：`Endpoint` 资源的一个可读名称。
- `project`：您的项目 ID。
- `location`：您的地区。
- `labels`：（可选）以键/值对的形式定义的用于 `Endpoint` 的用户元数据。

此方法返回一个 `Endpoint` 对象。

了解更多关于[Vertex AI Endpoints](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)。

In [ ]:
endpoint = aip.Endpoint.create(
    display_name="boston_" + UUID,
    project=PROJECT_ID,
    location=REGION,
    labels={"your_key": "your_value"},
)

print(endpoint)

部署 `Model` 资源到一个 `Endpoint` 资源。

您可以将一个或多个 `Vertex AI Model` 资源实例部署到同一个终端。每个部署的 `Vertex AI Model` 资源都会拥有自己的用于服务二进制文件的部署容器。

*注意：* 对于这个示例，您在上传模型工件到 `Vertex AI Model` 资源的上一步中指定了 TFHub 模型的部署容器。

在下一个示例中，您将把 `Vertex AI Model` 资源部署到一个 `Vertex AI Endpoint` 资源。`Vertex AI Model` 资源已经为其定义了部署容器镜像。要部署，您需要指定以下额外的配置设置：

- 机器类型。
- （如果有的话）GPU 的类型和数量。
- 静态、手动或自动扩展的 VM 实例。

在这个示例中，您将使用最少数量的指定参数部署模型，如下所示：

- `model`：`Model` 资源。
- `deployed_model_displayed_name`：部署模型实例的可读名称。
- `machine_type`：每个 VM 实例的机器类型。

由于需要为资源进行配置，可能需要几分钟来完成。

In [ ]:
response = endpoint.deploy(
    model=model,
    deployed_model_display_name="boston_" + UUID,
    machine_type=DEPLOY_COMPUTE,
)

print(endpoint)

### 创建测试项目

您可以使用`tf.keras.datasets`中的`load_data()`方法加载波士顿住房测试（保留）数据。该方法会返回一个包含两个元素的元组数据集。第一个元素是训练数据，第二个是测试数据。每个元素也是包含两个元素的元组：特征数据和相应的标签（自住房屋的中位数价值）。

您不需要训练数据，所以我们加载它为`(_,_)`。

在您可以用测试数据进行预测之前，您需要做以下操作：

`x_test`：
1. 通过将每个列中的数据除以该列的最大值来对数据进行标准化（重新缩放）。这将用介于0和1之间的32位浮点数替换每个单个值。

In [ ]:
import numpy as np
from tensorflow.keras.datasets import boston_housing

(_, _), (x_test, y_test) = boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
)


def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float32)
    return feature


# Let's save one data item that has not been scaled
x_test_notscaled = x_test[0:1].copy()

for _ in range(13):
    x_test[_] = scale(x_test[_])
x_test = x_test.astype(np.float32)

print(x_test.shape, x_test.dtype, y_test.shape)
print("scaled", x_test[0])
print("unscaled", x_test_notscaled)

In [ ]:
test_item = x_test[0]
test_label = y_test[0]
print(test_item.shape)

### 进行预测

现在你的 `Model` 资源已经部署到一个 `Endpoint` 资源中，可以通过向 `Endpoint` 资源发送预测请求来进行在线预测。

#### 请求

每个实例的格式为：

    [feature_list]

由于 explain() 方法可以接受多个实例，因此请将你的单个测试项作为一个测试项列表发送。

#### 响应

从 explain() 调用的响应是一个包含以下条目的 Python 字典：

- `ids`：每个预测请求的内部分配唯一标识符。
- `predictions`：每个实例的预测。
- `deployed_model_id`：进行预测的部署的 `Model` 资源的 Vertex AI 标识符。

In [ ]:
instances_list = [test_item.tolist()]

prediction = endpoint.predict(instances_list)
print(prediction)

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源：

In [ ]:
delete_bucket = False
delete_model = True
delete_endpoint = True
delete_batch_job = True

if delete_endpoint:
    try:
        endpoint.undeploy_all()
        endpoint.delete()
    except Exception as e:
        print(e)

if delete_model:
    try:
        model.delete()
    except Exception as e:
        print(e)

if delete_bucket:
    try:
        batch_predict_job.delete()
    except Exception as e:
        print(e)

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}